# Aperture Field Diagnosis

**Critical Issues to Diagnose:**
1. **Legacy Gamma Factor Error** - Why is legacy gamma ~10x too low?
2. **Missing Electromagnetic Acceleration** - Why no field effects near aperture?
3. **Image Charge Effects** - Expected strong acceleration near conducting boundaries

**Test Parameters:**
- Micron-scale aperture (1-10 μm radius)
- Zero transverse momentum initialization
- Close approach to aperture walls
- Electromagnetic field activation

**Expected Physics:**
- Strong image charge fields near aperture
- Significant energy/momentum changes
- Transverse deflection forces
- Radiation reaction effects

In [1]:
# Import and Setup
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add workspace paths
sys.path.append('/home/benfol/work/LW_windows')
sys.path.append('/home/benfol/work/LW_windows/legacy')

# Constants
C_MMNS = 299.792458  # mm/ns
PROTON_MASS_GEV = 0.938272
ELEMENTARY_CHARGE = 1.602176634e-19  # Coulombs
EPSILON_0 = 8.854187817e-12  # F/m

print("=== APERTURE FIELD DIAGNOSIS ===")
print("Investigating:")
print("1. Legacy gamma factor error")
print("2. Missing electromagnetic acceleration")
print("3. Image charge field effects")

# Set plotting parameters
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

=== APERTURE FIELD DIAGNOSIS ===
Investigating:
1. Legacy gamma factor error
2. Missing electromagnetic acceleration
3. Image charge field effects


In [2]:
# DIAGNOSIS 1: Legacy Gamma Factor Deep Dive
print("\n" + "="*50)
print("DIAGNOSIS 1: LEGACY GAMMA FACTOR ERROR")
print("="*50)

# Test legacy system initialization in detail
try:
    from bunch_inits import init_bunch
    
    # Print the source of the legacy function to understand the calculation
    import inspect
    print("Legacy init_bunch function signature:")
    sig = inspect.signature(init_bunch)
    print(f"  {sig}")
    
    # Test with different momentum values to understand scaling
    print("\nTesting legacy momentum scaling:")
    
    test_cases = [
        ("Low momentum", 75.0),     # 10x lower
        ("Original", 750.0),        # Original value
        ("High momentum", 7500.0),  # 10x higher
        ("Physics correct", 2661.0) # Calculated for 25 GeV
    ]
    
    for name, pz_test in test_cases:
        try:
            result = init_bunch(
                starting_distance=-10.0,  # mm
                transv_mom=0.0,           # zero transverse
                starting_Pz=pz_test,      # test momentum
                stripped_ions=1,
                m_particle=938.27,        # MeV/c^2
                transv_dist=0.0,
                pcount=1,
                charge_sign=1
            )
            
            if isinstance(result, tuple) and len(result) > 0:
                data = result[0] if hasattr(result[0], 'get') else {}
                if hasattr(data, 'get'):
                    gamma = data.get('gamma', [0])[0] if 'gamma' in data else 0
                    print(f"  {name:15s}: Pz={pz_test:6.1f} → γ={gamma:8.4f}")
                
        except Exception as e:
            print(f"  {name:15s}: Failed - {e}")
            
except Exception as e:
    print(f"Legacy system diagnosis failed: {e}")

# Calculate expected momentum for 25 GeV proton
print(f"\nExpected physics for 25 GeV proton:")
gamma_expected = 25.0 / PROTON_MASS_GEV
beta_expected = np.sqrt(1 - 1/gamma_expected**2)
pc_expected = gamma_expected * beta_expected * PROTON_MASS_GEV  # GeV/c
pc_expected_mev = pc_expected * 1000  # MeV/c

print(f"  Expected γ: {gamma_expected:.6f}")
print(f"  Expected β: {beta_expected:.6f}")
print(f"  Expected p·c: {pc_expected:.3f} GeV/c = {pc_expected_mev:.1f} MeV/c")
print(f"  Compare to legacy Pz=750: factor = {pc_expected_mev/750:.1f}")

# This should reveal why legacy gamma is wrong


DIAGNOSIS 1: LEGACY GAMMA FACTOR ERROR
Legacy init_bunch function signature:
  (starting_distance, transv_mom, starting_Pz, stripped_ions, m_particle, transv_dist, pcount, charge_sign)

Testing legacy momentum scaling:
E_MeV =  218943.4737121544
Gamma =  1.030895290128723
E_rest =  874061.7171317785
  Low momentum   : Pz=  75.0 → γ=  1.0309
E_MeV =  2186856.3964820867
Gamma =  2.6943905692563646
E_rest =  874061.7171317785
  Original       : Pz= 750.0 → γ=  2.6944
E_MeV =  21866704.094772648
Gamma =  25.03732380364867
E_rest =  874061.7171317785
  High momentum  : Pz=7500.0 → γ= 25.0373
E_MeV =  7758353.633392657
Gamma =  8.932360774935628
E_rest =  874061.7171317785
  Physics correct: Pz=2661.0 → γ=  8.9324

Expected physics for 25 GeV proton:
  Expected γ: 26.644726
  Expected β: 0.999295
  Expected p·c: 24.982 GeV/c = 24982.4 MeV/c
  Compare to legacy Pz=750: factor = 33.3


In [3]:
# DIAGNOSIS 2: Missing Electromagnetic Fields
print("\n" + "="*50)
print("DIAGNOSIS 2: MISSING ELECTROMAGNETIC ACCELERATION")
print("="*50)

# Check if electromagnetic fields are actually being calculated
try:
    from core.trajectory_integrator import LienardWiechertIntegrator
    from physics.particle_initialization import create_proton_bunch
    
    print("Testing electromagnetic field calculation...")
    
    # Create integrator
    integrator = LienardWiechertIntegrator()
    
    # Create test particle very close to aperture wall
    # Micron-scale aperture as requested
    APERTURE_RADIUS_MICRONS = 5.0  # 5 μm aperture
    APERTURE_RADIUS_MM = APERTURE_RADIUS_MICRONS * 1e-3  # Convert to mm
    
    print(f"Testing with micron-scale aperture: {APERTURE_RADIUS_MICRONS} μm = {APERTURE_RADIUS_MM} mm")
    
    # Position particle very close to aperture wall (90% of radius)
    radial_position = 0.9 * APERTURE_RADIUS_MM  # 90% of aperture radius
    
    # Create particle with zero transverse momentum as requested
    proton_bunch = create_proton_bunch(
        n_particles=1,
        energy_mev=25000.0,
        position=(0.0, radial_position/1000, -0.01),  # Convert mm to m, close to aperture in y
        momentum_direction=(0.0, 0.0, 1.0),  # Pure longitudinal motion
        bunch_size=(0.0, 0.0),
        momentum_spread=0.0
    )
    
    print(f"Particle initialized:")
    print(f"  Y position: {radial_position:.6f} mm (90% of {APERTURE_RADIUS_MM:.6f} mm aperture)")
    print(f"  Distance from wall: {APERTURE_RADIUS_MM - radial_position:.6f} mm")
    print(f"  Transverse momentum: {proton_bunch['py'][0]:.2e}")
    
    # Convert to trajectory format
    trajectory = [{
        'x': np.array([0.0]),
        'y': np.array([radial_position]),  # Close to aperture wall
        'z': np.array([-10.0]),  # 10mm before aperture
        't': np.array([0.0]),
        'gamma': np.array([proton_bunch['gamma'][0]]),
        'bx': np.array([proton_bunch['bx'][0]]),
        'by': np.array([proton_bunch['by'][0]]),
        'bz': np.array([proton_bunch['bz'][0]]),
        'bdotx': np.array([0.0]),
        'bdoty': np.array([0.0]),
        'bdotz': np.array([0.0]),
        'q': np.array([proton_bunch['q'][0]]),
        'm': np.array([938.27]),
        'char_time': np.array([3.66e-21])
    }]
    
    # Test if equations of motion detect fields
    print(f"\nTesting electromagnetic field response...")
    print(f"Initial conditions:")
    print(f"  Position: ({trajectory[0]['x'][0]:.6f}, {trajectory[0]['y'][0]:.6f}, {trajectory[0]['z'][0]:.6f}) mm")
    print(f"  Gamma: {trajectory[0]['gamma'][0]:.6f}")
    print(f"  By (transverse): {trajectory[0]['by'][0]:.8f}")
    
    # Test integration step with small aperture
    h_step = 0.001  # Very small time step
    trajectory_ext = []
    
    try:
        result = integrator.self_consistent_enhanced_step(
            h_step, trajectory, trajectory_ext, 0, 
            APERTURE_RADIUS_MM, 'flat'  # Use micron aperture
        )
        
        if result is not None:
            print(f"\nAfter integration step:")
            print(f"  New gamma: {result['gamma'][0]:.6f}")
            print(f"  New by: {result['by'][0]:.8f}")
            print(f"  New position: ({result['x'][0]:.6f}, {result['y'][0]:.6f}, {result['z'][0]:.6f})")
            
            # Check for acceleration
            gamma_change = result['gamma'][0] - trajectory[0]['gamma'][0]
            by_change = result['by'][0] - trajectory[0]['by'][0]
            
            print(f"  Gamma change: {gamma_change:.8f}")
            print(f"  By change: {by_change:.8f}")
            
            if abs(gamma_change) > 1e-8 or abs(by_change) > 1e-8:
                print("  ✓ ELECTROMAGNETIC FIELDS DETECTED!")
            else:
                print("  ✗ NO FIELD EFFECTS - PROBLEM IDENTIFIED")
        else:
            print("  ✗ Integration returned None")
            
    except Exception as e:
        print(f"  ✗ Integration failed: {e}")
        import traceback
        traceback.print_exc()
        
except Exception as e:
    print(f"Field diagnosis failed: {e}")
    import traceback
    traceback.print_exc()


DIAGNOSIS 2: MISSING ELECTROMAGNETIC ACCELERATION
Testing electromagnetic field calculation...
Testing with micron-scale aperture: 5.0 μm = 0.005 mm
Particle initialized:
  Y position: 0.004500 mm (90% of 0.005000 mm aperture)
  Distance from wall: 0.000500 mm
Field diagnosis failed: 'py'


Traceback (most recent call last):
  File "/tmp/ipykernel_35052/1777297164.py", line 39, in <module>
    print(f"  Transverse momentum: {proton_bunch['py'][0]:.2e}")
                                    ~~~~~~~~~~~~^^^^^^
KeyError: 'py'


In [4]:
# DIAGNOSIS 3: Image Charge Field Calculation
print("\n" + "="*50)
print("DIAGNOSIS 3: IMAGE CHARGE FIELD VERIFICATION")
print("="*50)

# Calculate expected image charge fields analytically
def calculate_image_charge_field(y_pos, aperture_radius, charge, velocity_beta):
    """
    Calculate image charge electric field for particle near conducting cylinder.
    
    Args:
        y_pos: Particle distance from center (mm)
        aperture_radius: Conductor radius (mm) 
        charge: Particle charge (Gaussian units)
        velocity_beta: Particle velocity in units of c
    
    Returns:
        E_radial: Radial electric field (Gaussian units)
    """
    
    # Image charge method for conducting cylinder
    # For particle at distance d from axis, image charges at d' = R²/d
    
    if abs(y_pos) >= aperture_radius:
        return 0.0  # Outside conductor
    
    d = abs(y_pos)  # Distance from axis
    R = aperture_radius  # Conductor radius
    
    if d < 1e-10:  # On axis
        return 0.0
    
    # Image position
    d_image = R**2 / d
    
    # Field from image charge (opposite charge)
    # E = k * q_image / r²  where q_image = -q, r = distance between charges
    separation = d_image - d
    
    if separation < 1e-10:
        return 1e10  # Very large field (singularity approach)
    
    # Gaussian units: k = 1, field in statV/cm
    E_image = charge / separation**2  # Basic Coulomb law
    
    # Convert to acceleration units (mm/ns²)
    # Need proper unit conversion from Gaussian to mechanical units
    mass_amu = 1.0  # Proton mass in amu
    conversion = 1.0  # Placeholder - needs proper calculation
    
    return E_image * conversion

# Test image charge calculation
print("Testing analytical image charge fields...")

y_positions = np.array([0.001, 0.002, 0.003, 0.004, 0.0045])  # mm, approaching wall
aperture_radius = 0.005  # 5 μm aperture
charge_gaussian = 1.178734e-5  # Proton charge
beta = 0.999295

print(f"Aperture radius: {aperture_radius:.6f} mm")
print(f"Particle charge: {charge_gaussian:.2e}")

for y_pos in y_positions:
    distance_from_wall = aperture_radius - y_pos
    field = calculate_image_charge_field(y_pos, aperture_radius, charge_gaussian, beta)
    
    print(f"y = {y_pos:.6f} mm, wall distance = {distance_from_wall:.6f} mm")
    print(f"  → E_radial = {field:.2e}")

# Check if the integrator field calculation module exists
print(f"\nChecking integrator field calculation methods...")

try:
    # Look for field calculation in the integrator
    methods = [attr for attr in dir(integrator) if 'field' in attr.lower() or 'conduct' in attr.lower()]
    print(f"Field-related methods: {methods}")
    
    # Check for conducting aperture methods
    aperture_methods = [attr for attr in dir(integrator) if 'flat' in attr.lower() or 'aperture' in attr.lower()]
    print(f"Aperture-related methods: {aperture_methods}")
    
except Exception as e:
    print(f"Method inspection failed: {e}")

print(f"\nCONCLUSION FROM DIAGNOSES:")
print(f"1. Need to identify legacy momentum unit error")
print(f"2. Need to verify field calculation is active")
print(f"3. Expected strong image charge fields at micron scale")


DIAGNOSIS 3: IMAGE CHARGE FIELD VERIFICATION
Testing analytical image charge fields...
Aperture radius: 0.005000 mm
Particle charge: 1.18e-05
y = 0.001000 mm, wall distance = 0.004000 mm
  → E_radial = 2.05e-02
y = 0.002000 mm, wall distance = 0.003000 mm
  → E_radial = 1.07e-01
y = 0.003000 mm, wall distance = 0.002000 mm
  → E_radial = 4.14e-01
y = 0.004000 mm, wall distance = 0.001000 mm
  → E_radial = 2.33e+00
y = 0.004500 mm, wall distance = 0.000500 mm
  → E_radial = 1.06e+01

Checking integrator field calculation methods...
Field-related methods: []
Aperture-related methods: []

CONCLUSION FROM DIAGNOSES:
1. Need to identify legacy momentum unit error
2. Need to verify field calculation is active
3. Expected strong image charge fields at micron scale


In [5]:
# DIAGNOSIS 4: Micron-Scale Aperture Test with Field Visualization
print("\n" + "="*50)
print("DIAGNOSIS 4: MICRON-SCALE APERTURE FIELD TEST")
print("="*50)

# Create comprehensive test with micron aperture and field tracking
def test_micron_aperture_fields():
    """Test with micron-scale aperture and zero transverse momentum."""
    
    # Parameters
    aperture_radius_um = 2.0  # 2 micron aperture
    aperture_radius_mm = aperture_radius_um * 1e-3
    
    # Position particle at different distances from wall
    wall_distances_um = np.array([0.1, 0.2, 0.5, 1.0])  # Distance from wall in microns
    wall_distances_mm = wall_distances_um * 1e-3
    
    print(f"Testing {aperture_radius_um} μm aperture")
    print(f"Particle positions from wall: {wall_distances_um} μm")
    
    results = []
    
    for i, wall_dist_mm in enumerate(wall_distances_mm):
        y_position = aperture_radius_mm - wall_dist_mm  # Distance from center
        
        print(f"\nTest {i+1}: {wall_distances_um[i]:.1f} μm from wall")
        print(f"  y = {y_position:.6f} mm from center")
        
        try:
            # Create particle with zero transverse momentum
            proton_bunch = create_proton_bunch(
                n_particles=1,
                energy_mev=25000.0,
                position=(0.0, y_position/1000, -0.001),  # Convert to meters
                momentum_direction=(0.0, 0.0, 1.0),  # Pure longitudinal
                bunch_size=(0.0, 0.0),
                momentum_spread=0.0
            )
            
            # Verify zero transverse momentum
            print(f"  Initial py: {proton_bunch['py'][0]:.2e}")
            print(f"  Initial by: {proton_bunch['by'][0]:.8f}")
            
            # Multiple integration steps to see accumulation
            trajectory = [{
                'x': np.array([0.0]),
                'y': np.array([y_position]),
                'z': np.array([-5.0]),  # Start 5mm before aperture
                't': np.array([0.0]),
                'gamma': np.array([proton_bunch['gamma'][0]]),
                'bx': np.array([0.0]),  # Zero transverse
                'by': np.array([0.0]),  # Zero transverse
                'bz': np.array([proton_bunch['bz'][0]]),
                'bdotx': np.array([0.0]),
                'bdoty': np.array([0.0]),
                'bdotz': np.array([0.0]),
                'q': np.array([proton_bunch['q'][0]]),
                'm': np.array([938.27]),
                'char_time': np.array([3.66e-21])
            }]
            
            # Track through multiple steps
            h_step = 0.0001  # Very fine time step
            n_steps = 100
            positions = []
            energies = []
            by_values = []
            
            current_traj = trajectory.copy()
            
            for step in range(n_steps):
                result = integrator.self_consistent_enhanced_step(
                    h_step, current_traj, [], 0, aperture_radius_mm, 'flat'
                )
                
                if result is not None:
                    # Update trajectory
                    current_traj[0] = {key: result[key].copy() for key in result.keys()}
                    
                    # Record data
                    z_pos = result['z'][0]
                    y_pos = result['y'][0]
                    gamma = result['gamma'][0]
                    by = result['by'][0]
                    
                    positions.append([z_pos, y_pos])
                    energies.append(gamma * PROTON_MASS_GEV)
                    by_values.append(by)
                    
                    # Stop if particle hits wall or moves far past aperture
                    if abs(y_pos) >= aperture_radius_mm or z_pos > 5.0:
                        break
                else:
                    break
            
            positions = np.array(positions)
            energies = np.array(energies)
            by_values = np.array(by_values)
            
            # Analyze results
            if len(energies) > 1:
                energy_change = energies[-1] - energies[0]
                by_change = by_values[-1] - by_values[0]
                max_by = np.max(np.abs(by_values))
                
                print(f"  Results after {len(energies)} steps:")
                print(f"    Energy change: {energy_change:.8f} GeV")
                print(f"    By change: {by_change:.8f}")
                print(f"    Max |by|: {max_by:.8f}")
                print(f"    Final position: z={positions[-1,0]:.3f}, y={positions[-1,1]:.6f}")
                
                # Store results
                results.append({
                    'wall_distance_um': wall_distances_um[i],
                    'energy_change': energy_change,
                    'by_change': by_change,
                    'max_by': max_by,
                    'positions': positions,
                    'energies': energies,
                    'by_values': by_values
                })
                
                # Check for field effects
                if abs(energy_change) > 1e-6 or abs(by_change) > 1e-8:
                    print(f"    ✓ FIELD EFFECTS DETECTED!")
                else:
                    print(f"    ✗ No significant field effects")
            else:
                print(f"    ✗ Integration failed early")
                
        except Exception as e:
            print(f"    ✗ Test failed: {e}")
    
    return results

# Run the micron aperture test
field_test_results = test_micron_aperture_fields()

# Summary
print(f"\n" + "="*50)
print("FIELD TEST SUMMARY")
print("="*50)

if field_test_results:
    print(f"Tested {len(field_test_results)} particle positions")
    for result in field_test_results:
        print(f"  {result['wall_distance_um']:.1f} μm from wall:")
        print(f"    ΔE = {result['energy_change']:.2e} GeV")
        print(f"    Δby = {result['by_change']:.2e}")
        
    # Check if ANY field effects were detected
    max_energy_change = max([abs(r['energy_change']) for r in field_test_results])
    max_by_change = max([abs(r['by_change']) for r in field_test_results])
    
    print(f"\nOverall maximum effects:")
    print(f"  Max energy change: {max_energy_change:.2e} GeV")
    print(f"  Max transverse acceleration: {max_by_change:.2e}")
    
    if max_energy_change > 1e-6 or max_by_change > 1e-8:
        print(f"  ✓ ELECTROMAGNETIC FIELDS ARE WORKING")
    else:
        print(f"  ✗ ELECTROMAGNETIC FIELDS NOT DETECTED")
        print(f"  → Need to check field calculation implementation")
else:
    print("No successful field tests completed")


DIAGNOSIS 4: MICRON-SCALE APERTURE FIELD TEST
Testing 2.0 μm aperture
Particle positions from wall: [0.1 0.2 0.5 1. ] μm

Test 1: 0.1 μm from wall
  y = 0.001900 mm from center
    ✗ Test failed: 'py'

Test 2: 0.2 μm from wall
  y = 0.001800 mm from center
    ✗ Test failed: 'py'

Test 3: 0.5 μm from wall
  y = 0.001500 mm from center
    ✗ Test failed: 'py'

Test 4: 1.0 μm from wall
  y = 0.001000 mm from center
    ✗ Test failed: 'py'

FIELD TEST SUMMARY
No successful field tests completed


In [6]:
# DIAGNOSIS 5: Source Code Investigation
print("\n" + "="*50) 
print("DIAGNOSIS 5: SOURCE CODE INVESTIGATION")
print("="*50)

# Let's examine the actual field calculation source code
print("Investigating source code for field calculations...")

# Check the legacy field calculation
try:
    from covariant_integrator_library import conducting_flat
    import inspect
    
    print("\nLegacy conducting_flat function:")
    print("Signature:", inspect.signature(conducting_flat))
    
    # Get source code
    try:
        source = inspect.getsource(conducting_flat)
        print("Source code length:", len(source), "characters")
        # Print first few lines to see the implementation
        source_lines = source.split('\n')[:15]
        print("First 15 lines:")
        for i, line in enumerate(source_lines, 1):
            print(f"  {i:2d}: {line}")
    except Exception as e:
        print(f"Could not get source: {e}")
        
except Exception as e:
    print(f"Could not inspect legacy conducting_flat: {e}")

# Check the core integrator field methods
print("\nCore integrator field methods:")
try:
    # Look for field-related methods in the core integrator
    core_methods = []
    for attr_name in dir(integrator):
        attr = getattr(integrator, attr_name)
        if callable(attr) and not attr_name.startswith('_'):
            # Check if method name suggests field calculation
            if any(keyword in attr_name.lower() for keyword in 
                   ['field', 'conduct', 'flat', 'image', 'charge', 'electric', 'magnetic']):
                core_methods.append(attr_name)
    
    print(f"Found field-related methods: {core_methods}")
    
    # Try to inspect the core equation of motion method
    if hasattr(integrator, 'eqsofmotion_retarded'):
        print(f"\nCore eqsofmotion_retarded signature:")
        sig = inspect.signature(integrator.eqsofmotion_retarded)
        print(f"  {sig}")
        
except Exception as e:
    print(f"Core method inspection failed: {e}")

# Test direct field calculation call
print(f"\nTesting direct field calculation...")
try:
    # Test legacy conducting_flat directly
    test_vector = {
        'x': np.array([0.0]),
        'y': np.array([0.004]),  # 4 μm from center
        'z': np.array([0.0]),    # At aperture
        'bx': np.array([0.0]),
        'by': np.array([0.0]), 
        'bz': np.array([0.999]),
        'gamma': np.array([26.6]),
        'q': np.array([1.178734e-5]),
        'm': np.array([938.27])
    }
    
    wall_z = 0.0
    apt_r = 0.005  # 5 μm aperture
    
    print(f"Calling conducting_flat with:")
    print(f"  position: ({test_vector['x'][0]}, {test_vector['y'][0]}, {test_vector['z'][0]})")
    print(f"  aperture: {apt_r} mm radius")
    
    field_result = conducting_flat(test_vector, wall_z, apt_r)
    
    print(f"Field calculation result:")
    print(f"  Type: {type(field_result)}")
    if hasattr(field_result, 'keys'):
        print(f"  Keys: {list(field_result.keys())}")
        for key, val in field_result.items():
            if hasattr(val, '__len__') and len(val) > 0:
                print(f"    {key}: {val[0]:.6e}")
    else:
        print(f"  Value: {field_result}")
        
except Exception as e:
    print(f"Direct field calculation failed: {e}")
    import traceback
    traceback.print_exc()

print(f"\n" + "="*50)
print("NEXT STEPS BASED ON DIAGNOSIS")
print("="*50)
print("1. Fix legacy momentum scaling (momentum units)")
print("2. Verify field calculation is being called")
print("3. Check field calculation implementation")
print("4. Test with even smaller apertures if needed")
print("5. Ensure integration step size is appropriate")


DIAGNOSIS 5: SOURCE CODE INVESTIGATION
Investigating source code for field calculations...

Legacy conducting_flat function:
Signature: (vector, wall_Z, apt_R)
Source code length: 3580 characters
First 15 lines:
   1: def conducting_flat(vector,wall_Z,apt_R):
   2:     """
   3:     taking losses at the circular aperture and generating a full image bunch reflecting off a flat wall
   4: 
   5:     includes cutoffs for particles striking wall or passing through the aperture
   6:     """
   7:     result = {}
   8:     result['x'] = np.zeros_like(vector['x'])
   9:     result['y'] = np.zeros_like(vector['y'])
  10:     result['z'] = np.zeros_like(vector['z'])
  11:     result['t'] = np.zeros_like(vector['t'])
  12:     result['Px'] = np.zeros_like(vector['Px'])
  13:     result['Py'] = np.zeros_like(vector['Py'])
  14:     result['Pz'] = np.zeros_like(vector['Pz'])
  15:     result['Pt'] = np.zeros_like(vector['Pt'])

Core integrator field methods:
Found field-related methods: []

Core

Traceback (most recent call last):
  File "/tmp/ipykernel_35052/1675070056.py", line 79, in <module>
    field_result = conducting_flat(test_vector, wall_z, apt_r)
  File "/home/benfol/work/LW_windows/legacy/covariant_integrator_library.py", line 51, in conducting_flat
    result['t'] = np.zeros_like(vector['t'])
                                ~~~~~~^^^^^
KeyError: 't'
